# CommonLit Readability (Kaggle Challenge)
## Training with BERT

TODO:
- try dataset with collator like in Huggingface course
- is BertForSequenceClassification really the appropriate model to use for regression?

In [1]:
import pandas as pd
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.utils.data import (
    Dataset,
    DataLoader
)
from transformers import (
    AutoTokenizer,
    BertForSequenceClassification,
    AdamW,
    get_linear_schedule_with_warmup
)

from utils.train_with_bert import BERTDataset, training, validating

TOKENIZERS_PARALLELISM = False

### Preprocess data

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# get max length of input ids in train_df
text_length = list()
for text in train_df["excerpt"]:
    tokens = tokenizer.encode(text)
    text_length.append(len(tokens))

max_length = max(text_length)
print(max_length)

314


### Splitting the dataset

In [4]:
train_df["kfold"] = train_df.index % 5

train_fold = train_df[train_df["kfold"] != 0].reset_index(drop=True)
valid_fold = train_df[train_df["kfold"] == 0].reset_index(drop=True)

### Building Dataset and DataLoader

In [5]:
train_ds = BERTDataset(tokenizer, max_length, train_fold["excerpt"], train_fold["target"])
valid_ds = BERTDataset(tokenizer, max_length, valid_fold["excerpt"], valid_fold["target"])

train_bs = 8
valid_bs = 16

train_dl = DataLoader(
    train_ds, 
    batch_size=train_bs,
    shuffle=True,
    num_workers=2,
    pin_memory=False
)
valid_dl = DataLoader(
    valid_ds, 
    batch_size=valid_bs,
    shuffle=False,
    num_workers=2,
    pin_memory=False
)

### Model

In [6]:
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

model.to(device)
model.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

cuda


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

### Training function

In [7]:
optimizer = AdamW(model.parameters(), lr=0.005, weight_decay=0.001)

epochs = 20
train_steps = int(len(train_fold)/train_bs*epochs)
num_steps = int(train_steps * 0.1)

scheduler = get_linear_schedule_with_warmup(optimizer, num_steps, train_steps)

In [8]:
train_losses = list()
valid_losses = list()

for epoch in tqdm(range(epochs)):
    
    print("EPOCH", epoch + 1)
    
    train_loss = training(train_dl, model, optimizer, scheduler, device)
    valid_loss = validating(valid_dl, model, device)
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

  0%|          | 0/20 [00:00<?, ?it/s]

EPOCH 1
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 5.80 GiB total capacity; 4.17 GiB already allocated; 66.62 MiB free; 4.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
torch.cuda.empty_cache()